In [1]:
import pandas as pd
import torch
from torch import nn
import torchvision
from torchvision import transforms
from IPython import display
import random
from datetime import datetime
from sklearn.model_selection import TimeSeriesSplit

# Load data

In [3]:
normalised_data = pd.read_csv("./data/normalised_result.csv")

In [4]:
# Date = How many days since 2008-09-02?
# Date encoding this is a fairly slow transformation and you need let it run for a while
normalised_data['Date'] = normalised_data['Date'].apply(lambda x: int((datetime.strptime(x, '%Y-%m-%d')-datetime.strptime('2008-09-02', '%Y-%m-%d')).total_seconds()/60/60/24))
#Convert to tensor
np_array = normalised_data.to_numpy()
tensor_data = torch.tensor(normalised_data.values)

In [39]:
# Generate forward chaining cross-validation ranges
tscv = TimeSeriesSplit()
split_indices = tscv.split(normalised_data)
train_indice_ranges = []
test_indice_ranges = []
for train_index, test_index in split_indices:
    train_indice_ranges.append((0,train_index[len(train_index)-1]))
    test_indice_ranges.append((test_index[0],test_index[len(test_index)-1]))

In [40]:
#Initialize data loaders
batch_size = 256
data_loaders = []
for fold in range(len(train_indice_ranges)):
    train_range = train_indice_ranges[fold]
    test_range = test_indice_ranges[fold]
    training_iter = torch.utils.data.DataLoader(tensor_data[test_range[0]:test_range[1]], batch_size, shuffle=False,num_workers=2)         
    testing_iter = torch.utils.data.DataLoader(tensor_data[train_range[0]:train_range[1]], batch_size, shuffle=False,num_workers=2)
    data_loaders.append((training_iter,testing_iter))

# Model

In [29]:
# Hyperparameters
wd = 0.0005
lr = 0.1

In [31]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()

    def forward(self,x):
        return x

# Intialization

In [32]:
model = Net()

In [34]:
def init_weights(m):
    if type(m) == nn.Linear: # by checking thr type we can init different layers in different ways
        torch.nn.init.xavier_uniform_(m.weight)    

model.apply(init_weights);

# Loss and optimizer


In [35]:
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), weight_decay=wd, lr=lr)

ValueError: optimizer got an empty parameter list

# Evaluation


In [36]:
def accuracy(y_hat, y):  #y_hat is a matrix; 2nd dimension stores prediction scores for each class.
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        y_hat = y_hat.argmax(axis=1) # Predicted class is the index of max score         
    cmp = (y_hat.type(y.dtype) == y)  # because`==` is sensitive to data types
    return float(torch.sum(cmp)) # Taking the sum yields the number of correct predictions.

# Example: only 1 sample is correctly classified.
y = torch.tensor([0, 2])
y_hat = torch.tensor([[0.1, 0.3, 0.6], [0.3, 0.2, 0.5]])
accuracy(y_hat, y) / len(y)

0.5

# Suplementary functions


In [ ]:
# Defined in file: ./chapter_preliminaries/calculus.md
def use_svg_display():  
    """Use the svg format to display a plot in Jupyter."""
    display.set_matplotlib_formats('svg')

# Defined in file: ./chapter_preliminaries/calculus.md
def set_axes(axes, xlabel, ylabel, xlim, ylim, xscale, yscale, legend):
    """Set the axes for matplotlib."""
    axes.set_xlabel(xlabel)
    axes.set_ylabel(ylabel)
    axes.set_xscale(xscale)
    axes.set_yscale(yscale)
    axes.set_xlim(xlim)
    axes.set_ylim(ylim)
    if legend:
        axes.legend(legend)
    axes.grid()

class Animator:  
    """For plotting data in animation."""
    def __init__(self, xlabel=None, ylabel=None, legend=None, xlim=None,
                 ylim=None, xscale='linear', yscale='linear',
                 fmts=('-', 'm--', 'g-.', 'r:'), nrows=1, ncols=1,
                 figsize=(3.5, 2.5)):
        # Incrementally plot multiple lines
        if legend is None:
            legend = []
        use_svg_display()
        self.fig, self.axes = mu.plt.subplots(nrows, ncols, figsize=figsize)
        if nrows * ncols == 1:
            self.axes = [self.axes, ]
        # Use a lambda function to capture arguments
        self.config_axes = lambda: set_axes(
            self.axes[0], xlabel, ylabel, xlim, ylim, xscale, yscale, legend)
        self.X, self.Y, self.fmts = None, None, fmts

    def add(self, x, y):
        # Add multiple data points into the figure
        if not hasattr(y, "__len__"):
            y = [y]
        n = len(y)
        if not hasattr(x, "__len__"):
            x = [x] * n
        if not self.X:
            self.X = [[] for _ in range(n)]
        if not self.Y:
            self.Y = [[] for _ in range(n)]
        for i, (a, b) in enumerate(zip(x, y)):
            if a is not None and b is not None:
                self.X[i].append(a)
                self.Y[i].append(b)
        self.axes[0].cla()
        for x, y, fmt in zip(self.X, self.Y, self.fmts):
            self.axes[0].plot(x, y, fmt)
        self.config_axes()
        display.display(self.fig)
        display.clear_output(wait=True)

In [ ]:
class Accumulator:  
    """For accumulating sums over `n` variables."""
    def __init__(self, n):
        self.data = [0.0] * n # [0, 0, ..., 0]
    def add(self, *args):
        self.data = [a + float(b) for a, b in zip(self.data, args)]
    def reset(self):
        self.data = [0.0] * len(self.data)
    def __getitem__(self, idx):
        return self.data[idx]

# Training

In [44]:
def train_epoch(net, data, optimizer, batch_size=256, num_outputs=10):  
    """The training function for one epoch."""
    # Set the model to training mode
    if isinstance(net, torch.nn.Module):
        net.train()

    # Metrics
    train_metric = Accumulator(3)
    test_metric = Accumulator(2)
        
    # Go over each fold
    for fold in data:
        train_iter = fold[0]  
        test_iter = fold[1]
        #Train
        for X in train_iter:
            y_hat = net(X)
            y = None # no ground truth / label
            l = loss(y_hat, y)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
            train_metric.add(float(l) * len(y), accuracy(y_hat, y), y.size().numel())
        #Evaluate
        for X in test_iter:
            for _, (X, y) in enumerate(test_iter):
                test_metric.add(accuracy(net(X), y), y.numel())
    
    # Return training loss, training accuracy, and test accuracy
    return train_metric[0] / train_metric[2], train_metric[1] / train_metric[2], test_metric[0] / test_metric[1]

In [45]:
def train(net, data, loss, num_epochs, updater): 
    animator = Animator(xlabel='epoch', xlim=[1, num_epochs], ylim=[0.0, 1.0],legend=['train loss', 'train acc', 'test acc'])
    for epoch in range(num_epochs):
        metrics = train_epoch(net, data, loss, updater)
        animator.add(epoch + 1, metrics)
    train_loss, train_acc, test_acc = metrics
    print(train_loss, train_acc, test_acc)
    assert train_loss < 0.5, train_loss
    assert train_acc <= 1 and train_acc > 0.7, train_acc
    assert test_acc <= 1 and test_acc > 0.7, test_acc

In [46]:
num_epochs = 30
train(model, data_loaders, loss, num_epochs, optimizer)

NameError: name 'train_iter' is not defined